# Tarbimisandmete kategoriseerimine ja sündmuste tuvastamine

## Andmete laadimine
___
Käesolev töö kasutab andmete kogumiseks Nordic Automation Systems-i (NAS) sensoreid (http://www.nasys.no/products/lora-water-meter-sensor/) ning neid andmeid kogutakse kasutades nende API-t (https://iothub.nasys.no). Andmed on kogutud juba sensori tasemel kumulatiivselt, liites iga impulsi varem alustatud loendurile juurde. Impulss on veemõõtja tiiviku täispööre. Iga nelja minuti tagant saadetakse loenduri näit välja NAS-i serverisse kui 9 baidine baidijada, kus esimesed neli baiti on LSB (least significant bit) formaadis täisarvuline loenduri väärtus. NAS-i API poole pöördudes tagastatakse JSON objekt, mille ühel väljal asub 9 baidine näit base64 enkodeeritud tekstiväärtusena.

In [20]:
from urllib.request import urlopen
from mainlib import get_nas_url
import json

jsonurl = urlopen(get_nas_url(response_count=3))
response = json.loads(jsonurl.read())
print(json.dumps(response, indent=4, sort_keys=True))

{
    "result": [
        {
            "adr": "true",
            "bandwidth": "0",
            "codeRate": "",
            "data": "ACUAEDU=",
            "date": "2017-02-17 18:36:51",
            "fcnt": "1",
            "fport": "99",
            "frequency": "0",
            "gatewaycount": "1",
            "id": "885781",
            "modulation": null,
            "rssi": "-91",
            "spreadFactor": "0",
            "timestamp": "2147483647"
        },
        {
            "adr": "true",
            "bandwidth": "0",
            "codeRate": "",
            "data": "AAcAAAAVlJYA",
            "date": "2017-02-17 18:41:09",
            "fcnt": "2",
            "fport": "24",
            "frequency": "0",
            "gatewaycount": "1",
            "id": "885805",
            "modulation": null,
            "rssi": "-103",
            "spreadFactor": "0",
            "timestamp": "2147483647"
        },
        {
            "adr": "true",
            "bandwidth": "0",
  

Käesolev töö kasutab peamiselt välju _data_ ning _date_.
___

### Andmete dekodeerimine
Andmete dekodeerimiseks on defineeritud kaks meetodit. Meetod water_meter_data_parser dekodeerib 9 baidise base64 stringi 4 baidiseks täisarvuliseks väärtuseks ning map_nas_data muundab JSON-ist tulnud _data_ välja reaalseks numbriliseks veenäiduks ning kogu data kirjutatakse faili.

In [21]:
from urllib.request import urlopen
from mainlib import file_name
import json
import base64


def water_meter_data_parser(nas_data_str):
    print(nas_data_str)
    btea_orig = bytearray(nas_data_str, 'UTF-8')
    btea = base64.b64decode(btea_orig)
    return int.from_bytes(btea[:4], byteorder='little', signed=False)


def map_nas_data(row):
    data_int = float(water_meter_data_parser(row['data']))
    return {
        'data': data_int / 100, # to l
        'date': row['date']
    }

usage_data_unparsed = response['result']
usage_data_list = list(map(map_nas_data, 
                                filter(lambda row: row['fport'] == '24', usage_data_unparsed)))

file = open(file_name, "w")
file.write(json.dumps(usage_data_list, indent=4, separators=(',', ': ')))
file.flush()
file.close()
print(json.dumps(usage_data_list, indent=4, sort_keys=True))

ACUAEDU=
AAcAAAAVlJYA
LAkAAAATAJYA
[
    {
        "data": 17.92,
        "date": "2017-02-17 18:41:09"
    },
    {
        "data": 23.48,
        "date": "2017-02-17 18:46:11"
    }
]
